# Homework 2
By Wanyan (Wendy) Shao

## Reading
* Descriptor: https://realpython.com/python-descriptors/#how-attributes-are-accessed-with-the-lookup-chain
* Data model: https://docs.python.org/3/reference/datamodel.html (contents related to what we taught)
* Python MRO: https://www.python.org/download/releases/2.3/mro/

## Merge sorted stream
Write a generator function that takes two sorted streams (generators), and return a generator that can produce a merged stream in sorted order.

Bonus point: can you make it generic such that it can merge any number of streams?


```python
def merge_sorted_stream(stream1, stream2):
    yield ...

stream1 = range(0, 10, 2)
stream2 = range(1, 10, 2)

for x in merge_sorted_stream(stream1, stream2):
    print(x)

0 
1
2
3
4
5
6
7
8
9
```

In [27]:
#compare the two streams by using two pointers
def merge_sorted_stream(stream1, stream2):
    n1 = len(stream1)
    n2 = len(stream2)
    # initialize two pointers
    p1 = 0
    p2 = 0
    
    # compare the two arrays by using two pointers
    # return the smallest one and move the corresponding pointer one step further
    while p1 < n1 and p2 < n2:
        if stream1[p1] <= stream2[p2]:
            yield stream1[p1]
            p1 += 1
        else:
            yield stream2[p2]
            p2 += 1
    
    # if there are still elements left
    if p1 < n1:
        for i in range(p1, n1, 1):
            yield stream1[i]
    if p2 < n2:
        for i in range(p2, n2, 1):
            yield stream2[i]

In [28]:
stream1 = range(0, 10, 2)
stream2 = range(1, 10, 2)

for x in merge_sorted_stream(stream1, stream2):
    print(x)

0
1
2
3
4
5
6
7
8
9


In [155]:
from itertools import chain

def merge_sorted_stream_general(*streams):# take (stream1, stream2, stream3,...) as argument
    generator_total = sorted(chain(*streams))# combin all the input streams into a new stream and sort the new stream
    for item in generator_total: # print items in the new stream
        yield item
    

In [156]:
stream1 = range(0, 10, 2)
stream2 = range(1, 10, 2)
stream3 = (x for x in [0.5,2.5,3.5,3.5,4.5,9.5] )
for x in merge_sorted_stream_general(stream1, stream2, stream3):
    print(x)

0
0.5
1
2
2.5
3
3.5
3.5
4
4.5
5
6
7
8
9
9.5


## Tree traversal

Unlike linear data structures (Array, Linked List, Queues, Stacks, etc) which have only one logical way to traverse them, trees can be traversed in different ways. Following are the generally used ways for traversing trees.

```
      1
    /  \ 
   2    3
  / \
 4   5
```

Depth First Traversals: 
  * (a) Inorder (Left, Root, Right) : 4 2 5 1 3
  * (b) Preorder (Root, Left, Right) : 1 2 4 5 3
  * (c) Postorder (Left, Right, Root) : 4 5 2 3 1

Define a Tree class with a method that can walk through the tree in different orders. Hint: use generator will make your life a lot easier.

```python
class TreeNode:
    
    ...

    def in_order(self):
        pass
        
    def pre_order(self):
        pass
        
    def post_order(self):
        pass

    
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

>>> print(' -> '.join(item for item in root.in_order()))
4 -> 2 -> 5 -> 1 -> 3    


```
   

In [47]:
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None
        
    def in_order(self):
        # use recursion method to implement in_order tranversal
        # first we tranverse left node, then root node, finally right node
        if self:
            # if the node is not NULL
            if self.left:
                for num in self.left.in_order():
                    yield str(num)
            yield str(self.val)
            if self.right:
                for num in self.right.in_order():
                    yield str(num)

            
        
    
    def pre_order(self):
         # use recursion method to implement in_order tranversal
        # first we tranverse root node, then left node, finally right node
        if self:
            yield str(self.val)
            if self.left:
                for num in self.left.pre_order():
                    yield str(num)
            if self.right:
                for num in self.right.pre_order():
                    yield str(num)
    
    def post_order(self):
         # use recursion method to implement in_order tranversal
        # first we tranverse left node, then right node, finally root node
        if self:
            if self.left:
                for num in self.left.post_order():
                    yield str(num)
            if self.right:
                for num in self.right.post_order():
                    yield str(num)
            yield str(self.val)
    
    

In [50]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

print("In_order tranversal:")
print(' -> '.join(item for item in root.in_order()))
print("Pre_order tranversal:")
print(' -> '.join(item for item in root.pre_order()))
print("Post_order tranversal:")
print(' -> '.join(item for item in root.post_order()))

In_order tranversal:
4 -> 2 -> 5 -> 1 -> 3
Pre_order tranversal:
1 -> 2 -> 4 -> 5 -> 3
Post_order tranversal:
4 -> 5 -> 2 -> 3 -> 1


## Implement a timer
Implement a timer that can print the execution time of your code. Try to implement it both as a decorator and as a context manager to compare the implementations. Can you implement it using one single class? 

Example:
```python
import time


with timer() as timer:
    time.sleep(3)

# Total execution time: 3.000123 seconds (a made up number)


@timer
def sleep(secs):
    time.sleep(secs)
    
sleep(3)
# Total execution time: 3.000123 seconds (a made up number)
```

Below is the code snippet to measure the time:
```python    
import time
start_time = time.time()
main()
print(f"--- {time.time() - start_time} seconds ---"
```

In [144]:
import time

# The class contains two parts:
# 1. using __enter__ and __exit to calclulate the duration time in the "with" block
# 2. using __call__ dunder method to work as a decorator 
# Each part needs a separate system to calculate the duration time
class timer:
    def __enter__(self):
        self.start_time = time.time()
        return self

    
    def __call__(self, func):
        def inner(secs):
            self.start_time = time.time()
            print("Execution...")
            func(secs)
            self.end_time = time.time()
            print(f"--- {self.end_time - self.start_time} seconds ---")
        return inner
    
    def __exit__(self, exc_type, exc_value ,traceback):
        self.end_time = time.time()
        print(f"--- {self.end_time - self.start_time} seconds ---")
        return True 

In [146]:
with timer() as timer:
    time.sleep(3)

--- 3.000107765197754 seconds ---


In [147]:
@timer
def sleep(secs):
    time.sleep(secs)


sleep(3)

Execution...
--- 3.001291275024414 seconds ---
